In [146]:
import openai


def get_mistral_rubra_response(prompt, model, functions, msgs):
  openai.api_key = "abc"
  openai.base_url = "http://localhost:8019/v1/"
  
  try:
    completion = openai.chat.completions.create(
      model=model,
      temperature=0.1,
      messages=msgs,
      functions=functions,
      function_call="auto",
      stream=False,
    )
    return completion.choices[0]
  except Exception as e:
    print(e, model, prompt)


In [161]:
system_prompt = "You are a helpful assistant."
functions = [
   {
      "type": "function",
    "function": {
      "name": "getCurrentWeather",
      "description": "Get the weather in location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
          "unit": {"type": "string", "enum": ["c", "f"]}
        },
        "required": ["location"]
      }
    }
  },
   {    "type": "function",
     "function":
    {
        "name": "orderUmbrella",
        "description": "Do this to help user to order an umbrella online", 
        "parameters": {
            "type": "object",
            "properties": {
                "brand_name": {
                    "type": "string",
                    "description": "The name of the umbrella brand"
                }
            },
            "required": [
                "brand_name"
            ]
        }
    }},
]


user_query = "check the weather in boston"
msgs = [{"role": "system", "content":system_prompt} ,{"role": "user", "content": user_query}, {"role": "function", "content": '<<functions>>[getCurrentWeather(location="Boston)]'}, {"role": "observation", "content": "<<observation>>72 f, rainy."}
        ,
        {"role": "assistant", "content": "The current weather in Boston is 72 degrees Fahrenheit and it's raining. Would you like to order an umbrella?"},
         {"role": "user", "content": "yes pls"},
        ]
# msgs = [{"role": "system", "content":system_prompt} ,{"role": "user", "content": user_query}]

res = get_mistral_rubra_response(user_query, "gorilla-openfunctions-v2", functions=functions, msgs=msgs)
print(res.message.content)

 The current weather in Boston is 72 degrees Fahrenheit and it's raining.


In [160]:
system_prompt = "You are a helpful assistant."
functions = [
    {"function":
    {
        "name": "get_stock_price",
        "description": "Get the current stock price",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock symbol, e.g. AAPL, GOOG"
                }
            },
            "required": [
                "symbol"
            ]
        }
    }},
    {"function":{
        "name": "check_word_anagram",
        "description": "Check if two words are anagrams of each other",
        "parameters": {
            "type": "object",
            "properties": {
                "word1": {
                    "type": "string",
                    "description": "The first word"
                },
                "word2": {
                    "type": "string",
                    "description": "The second word"
                }
            },
            "required": [
                "word1",
                "word2"
            ]
        }
    }}
]

user_query = "What's the stock price of Tesla and Google?"

# 
msgs = [{"role": "system", "content":system_prompt} ,{"role": "user", "content": user_query}, {"role": "function", "content": '<<functions>>[get_stock_price(symbol="TSLA")], <<functions>>[get_stock_price(symbol="GOOG")]'}, {"role": "observation", "content": "<<observation>>170, 138"}]
# msgs = [{"role": "system", "content":system_prompt} ,{"role": "user", "content": user_query},]
res = get_mistral_rubra_response(user_query, "gorilla-openfunctions-v2", functions=functions, msgs=msgs)
print(res.message.content)

 The current stock price of Tesla (TSLA) is $170 and the current stock price of Google (GOOG) is $138.
